**Этап подготовки данных**

In [23]:
import os
import pandas as pd

Определим необходимые функции для парсинга файлов с текстами


In [5]:
def read_file(path):
    with open(path, 'r', encoding='utf-8') as f:
        text = f.read()
    return text

In [13]:
def data_parse(sample: str, sentiment: str, data_dir: str = 'aclImdb') -> pd.DataFrame:
    """
    Читает отзывы из файлов в указанной директории и собирает их в DataFrame.

    Args:
        sample: Название выборки (например, 'train' или 'test').
        sentiment: Направление сентимент анализа ('pos' для положительных отзывов и 'neg' для отрицательных).
        data_dir: Путь к основной директории с данными. По умолчанию 'aclImdb'.

    Returns:
        pd.DataFrame: DataFrame, содержащий идентификаторы, текст отзывов, рейтинги и метки положительных отзывов.
    """
    
    output_frame = {
        'id': [],
        'text': [],
        'rating': [],
        'positive': []
    }

    dir_path = f'../{data_dir}/{sample}/{sentiment}'
    
    if not os.path.exists(dir_path):
        print(f"Directory {dir_path} does not exist.")
        return pd.DataFrame()

    files = os.listdir(dir_path)
    for filename in files:
        rating = filename[filename.find('_') + 1: filename.find('.')]
        rev_id = filename[:filename.find('_')]

        file_path = os.path.join(dir_path, filename)
        text = read_file(file_path)

        output_frame['rating'].append(rating)
        output_frame['text'].append(text)
        output_frame['id'].append(rev_id)

    output_frame['positive'].extend(
        [1] * len(files) if sentiment == 'pos' else [0] * len(files)
    )
    
    return pd.DataFrame.from_dict(output_frame)

Объединим позитивные и негативные отзывы в одну выборку

In [14]:
train_pos_df = data_parse(sample = 'train', sentiment = 'pos')
train_neg_df = data_parse(sample = 'train', sentiment = 'neg')

In [17]:
train_df = pd.concat([train_pos_df, train_neg_df])

In [16]:
train_df

,id,text,rating,positive
0,0,Bromwell High is a cartoon comedy. It ran at t...,9,1
1,10000,Homelessness (or Houselessness as George Carli...,8,1
2,10001,Brilliant over-acting by Lesley Ann Warren. Be...,10,1
3,10002,This is easily the most underrated film inn th...,7,1
4,10003,This is not the typical Mel Brooks film. It wa...,8,1
...,...,...,...,...
12495,9998,"Towards the end of the movie, I felt it was to...",4,0
12496,9999,This is the kind of movie that my enemies cont...,3,0
12497,999,I saw 'Descent' last night at the Stockholm Fi...,3,0
12498,99,Some films that you pick up for a pound turn o...,1,0


Аналогично для тестовой выборки

In [18]:
test_pos_df = data_parse(sample = 'test', sentiment = 'pos')
test_neg_df = data_parse(sample = 'test', sentiment = 'neg')

In [19]:
test_df = pd.concat([test_pos_df, test_neg_df])

Сохраним DataFrame в CSV файл

In [22]:
train_df.to_csv('../data/train.csv', index=False, encoding='utf-8')
test_df.to_csv('../data/test.csv', index=False, encoding='utf-8')
